# W1M3 - ETL 프로세스 구현하기

In [2]:
import requests
from bs4 import BeautifulSoup

/Users/admin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
html = requests.get("https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29").text

In [4]:
soup = BeautifulSoup(html, 'html.parser')

In [50]:
gdp = soup.find("table", {"class": 'wikitable'}).find('tbody').find_all('tr')[3:]
gdp

[<tr>
 <td style="text-align:left"><span class="flagicon" style="display:inline-block;width:25px;text-align:left"><span class="mw-image-border" typeof="mw:File"><span><img alt="" class="mw-file-element" data-file-height="650" data-file-width="1235" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/23px-Flag_of_the_United_States.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/35px-Flag_of_the_United_States.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/46px-Flag_of_the_United_States.svg.png 2x" width="23"/></span></span></span> <a class="mw-redirect" href="/wiki/GDP_of_the_United_States" title="GDP of the United States">United States</a></td>
 <td>30,337,162</td>
 <td>2025</td>
 <td>27,360,935</td>
 <td>2023</td>
 <td>25,744,100</td>
 <td>2022
 </td></tr>,
 <tr>
 <td style="text-align:left"><span class="flagicon" style="display:inline-blo

In [16]:
gdp_imf = [[gdp_row.find_all('td')[i].text.strip() for i in range(2)] for gdp_row in gdp]
gdp_imf

[['United States', '30,337,162'],
 ['China', '19,534,894'],
 ['Germany', '4,921,563'],
 ['Japan', '4,389,326'],
 ['India', '4,271,922'],
 ['United Kingdom', '3,730,261'],
 ['France', '3,283,429'],
 ['Italy', '2,459,597'],
 ['Canada', '2,330,308'],
 ['Brazil', '2,307,162'],
 ['Russia', '2,195,708'],
 ['South Korea', '1,947,133'],
 ['Australia', '1,881,140'],
 ['Spain', '1,827,576'],
 ['Mexico', '1,817,818'],
 ['Indonesia', '1,492,618'],
 ['Turkey', '1,455,413'],
 ['Netherlands', '1,272,960'],
 ['Saudi Arabia', '1,136,580'],
 ['Switzerland', '999,604'],
 ['Poland', '915,451'],
 ['Taiwan', '814,438'],
 ['Belgium', '689,364'],
 ['Sweden', '638,780'],
 ['Ireland', '587,225'],
 ['Argentina', '574,201'],
 ['United Arab Emirates', '568,567'],
 ['Singapore', '561,725'],
 ['Austria', '559,218'],
 ['Israel', '550,905'],
 ['Thailand', '545,341'],
 ['Norway', '503,466'],
 ['Malaysia', '488,250'],
 ['Philippines', '471,516'],
 ['Vietnam', '468,400'],
 ['Iran', '463,747'],
 ['Bangladesh', '451,856'],

In [18]:
import pandas as pd

df_gdp = pd.DataFrame(gdp_imf)
df_gdp

,0,1
0,United States,"30,337,162"
1,China,"19,534,894"
2,Germany,"4,921,563"
3,Japan,"4,389,326"
4,India,"4,271,922"
...,...,...
204,Kiribati,311
205,Palau,308
206,Marshall Islands,305
207,Nauru,161


In [19]:
df_gdp.columns = ['country', 'gdp']
df_gdp

,country,gdp
0,United States,"30,337,162"
1,China,"19,534,894"
2,Germany,"4,921,563"
3,Japan,"4,389,326"
4,India,"4,271,922"
...,...,...
204,Kiribati,311
205,Palau,308
206,Marshall Islands,305
207,Nauru,161


In [37]:
df_gdp['gdp'] = df_gdp['gdp'].apply(lambda x : int(x.replace(",","")) if x != '—' else None)
df_gdp['gdp'] = round(df_gdp['gdp']/1000, 2)
df_gdp

,country,gdp
0,United States,30337.16
1,China,19534.89
2,Germany,4921.56
3,Japan,4389.33
4,India,4271.92
...,...,...
204,Kiribati,0.31
205,Palau,0.31
206,Marshall Islands,0.30
207,Nauru,0.16


In [47]:
df_gdp = df_gdp.sort_values('gdp', ascending = False)
df_gdp

,country,gdp
0,United States,30337.16
1,China,19534.89
2,Germany,4921.56
3,Japan,4389.33
4,India,4271.92
...,...,...
178,Greenland,NaN
184,Eritrea,NaN
185,Zanzibar,NaN
192,Sint Maarten,NaN


In [49]:
df_100B = df_gdp[df_gdp['gdp'] >= 100]
df_100B

,country,gdp
0,United States,30337.16
1,China,19534.89
2,Germany,4921.56
3,Japan,4389.33
4,India,4271.92
...,...,...
68,Uzbekistan,112.65
69,Guatemala,112.37
70,Oman,109.99
71,Bulgaria,108.42


In [63]:
import pycountry
import pycountry_convert as pc

alpha2 = pc.country_name_to_country_alpha2("United States")
con_code = pc.country_alpha2_to_continent_code(alpha2)
con_code

'NA'

In [64]:
df_100B['continent'] = df_100B['country'].apply(lambda x : pc.country_alpha2_to_continent_code(pc.country_name_to_country_alpha2(x)))
df_100B

/var/folders/nz/bxfn5lh14zv5j6hhqnwzzycw0000gn/T/ipykernel_752/394398851.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_100B['continent'] = df_100B['country'].apply(lambda x : pc.country_alpha2_to_continent_code(pc.country_name_to_country_alpha2(x)))


,country,gdp,continent
0,United States,30337.16,NA
1,China,19534.89,AS
2,Germany,4921.56,EU
3,Japan,4389.33,AS
4,India,4271.92,AS
...,...,...,...
68,Uzbekistan,112.65,AS
69,Guatemala,112.37,NA
70,Oman,109.99,AS
71,Bulgaria,108.42,EU


In [170]:
df_100B.groupby('continent').head(5).groupby('continent').mean('gdp').sort_values('gdp', ascending = False)

,gdp
continent,
NA,6946.500
AS,6327.178
EU,3318.112
OC,1072.030
SA,791.566
AF,285.184


In [89]:
import sqlite3

conn = sqlite3.connect('World_Economies.db')
df_100B.to_sql('Countries_by_GDP', conn, index = False)

72

In [ ]:
cur = conn.cursor()

In [90]:
pd.read_sql(
    """
    SELECT * FROM COUNTRIES_BY_GDP""", conn)

,country,gdp,continent
0,United States,30337.16,NA
1,China,19534.89,AS
2,Germany,4921.56,EU
3,Japan,4389.33,AS
4,India,4271.92,AS
...,...,...,...
67,Uzbekistan,112.65,AS
68,Guatemala,112.37,NA
69,Oman,109.99,AS
70,Bulgaria,108.42,EU


In [98]:
def exceed_100B() :
    conn = sqlite3.connect('World_Economies.db')
    cur = conn.cursor()
    df = pd.read_sql("SELECT * FROM Countries_by_GDP WHERE GDP > 100", conn)
    return df

In [179]:
def region_top5_average() :
    conn = sqlite3.connect('World_Economies.db')
    cur = conn.cursor()
    df = pd.read_sql(
        """SELECT AVG(GDP), CONTINENT
        FROM (SELECT * FROM 
        (SELECT *, ROW_NUMBER() OVER (PARTITION BY CONTINENT ORDER BY GDP DESC) AS RANK 
        FROM Countries_by_GDP) 
        WHERE RANK <=5)
        GROUP BY CONTINENT
        """, conn)
    return df

In [180]:
exceed_100B()

,country,gdp,continent
0,United States,30337.16,NA
1,China,19534.89,AS
2,Germany,4921.56,EU
3,Japan,4389.33,AS
4,India,4271.92,AS
...,...,...,...
67,Uzbekistan,112.65,AS
68,Guatemala,112.37,NA
69,Oman,109.99,AS
70,Bulgaria,108.42,EU


In [181]:
region_top5_average()

,AVG(GDP),continent
0,285.184,AF
1,6327.178,AS
2,3318.112,EU
3,6946.500,NA
4,1072.030,OC
5,791.566,SA


In [165]:
pd.read_sql(
        """SELECT *
        FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY CONTINENT ORDER BY GDP DESC) AS RANK 
        FROM Countries_by_GDP)
        WHERE RANK <= 5
        """, conn)

,country,gdp,continent,RANK
0,South Africa,418.05,AF,1
1,Egypt,380.04,AF,2
2,Algeria,264.27,AF,3
3,Nigeria,194.96,AF,4
4,Morocco,168.60,AF,5
5,China,19534.89,AS,1
6,Japan,4389.33,AS,2
7,India,4271.92,AS,3
8,South Korea,1947.13,AS,4
9,Indonesia,1492.62,AS,5


In [178]:
pd.read_sql(
        """SELECT AVG(GDP), CONTINENT
        FROM (SELECT * FROM 
        (SELECT *, ROW_NUMBER() OVER (PARTITION BY CONTINENT ORDER BY GDP DESC) AS RANK 
        FROM Countries_by_GDP) 
        WHERE RANK <=5)
        GROUP BY CONTINENT
        """, conn)

,AVG(GDP),continent
0,285.184,AF
1,6327.178,AS
2,3318.112,EU
3,6946.500,NA
4,1072.030,OC
5,791.566,SA
